# 制作评估语料

注意输入的 Context tokens 长度要小于最大模型最大生成序列长度的一半！

这个 notebook 针对的输入 corpus 文件是：

- 每个文档一行
- 文档包含段落数组
- 段落包含句子数组

的 Loose JSON 文件

## 代码准备

### Imporings

In [14]:
import csv
import json
import math
import os
import sys
import random
from contextlib import ExitStack
from datetime import timedelta
from functools import partial
from glob import glob, iglob
from itertools import chain, cycle, islice, count, accumulate
from multiprocessing import Pool
from time import time

import numpy as np
import sentencepiece as spm
from tqdm.auto import tqdm


### Constants

In [15]:
MAX_LEN = 128
MIN_LEN = 16

SPM_MODEL = '../data/spm/gpt2_huamei_corpus_bpe_32k_v2.model'

SP = spm.SentencePieceProcessor()
SP.load(SPM_MODEL)

True

### Functions

In [16]:
def text_files_line_iterator(paths):
    return chain.from_iterable(
        open(path)
        for path
        in tqdm(paths, '[iter files]', unit='file')
    )


def single_text_file_line_count(path, show_progress_bar=False):
    with open(path) as fd:
        iterable = tqdm(fd) if show_progress_bar else fd
        return sum(1 for _ in iterable)
        

def text_files_line_count(paths):
    try:
        total = len(paths)
    except (AttributeError, TypeError):
        total = None
    with Pool() as pool:
        it = pool.imap_unordered(
            single_text_file_line_count,
            tqdm(paths, '[map: files]', unit='file')
        )
        return sum(c for c in tqdm(it, '[reduce: sum lines]', unit='file', total=total))


def proc_line(line):
    result = []
    line = line.strip()
    if not line:
        return result
    paragraphs = json.loads(line)
    text = ''
    n_text = 0
    for sentence in chain.from_iterable(paragraphs):
        sentence = sentence.strip()
        if not sentence:
            continue
        n_sentence = len(SP.encode_as_ids(sentence))
        if n_text + n_sentence < SEQ_LENGTH + MIN_CTX_LEN // 2:
            text += sentence
            n_text += n_sentence
        else:
            result.append({'text': text, 'length': n_text})
            text = sentence
            n_text = n_sentence
    if n_text:
        result.append({'text': text, 'length': n_text})
    return result
    

## 语料文件

### 输入文件

### 列出输入文件

In [17]:
# INPUT_FILES = [
#     path
#     for path in tqdm(iglob(
#         "/nfs/server01_public/豆瓣/情感相关的小组/data.json/*",
#         recursive=True
#     ))
#     if os.path.isfile(path) and os.path.splitext(path)[1].lower() in ('json', '.jsonl', '.jsonlines', 'json', 'jsonline')
# ]

INPUT_FILES = [
    '/nfs/server01_public/data/gpt2/output/xinli_20191016.jsonl'
]

print(f'源语料文件数：{len(INPUT_FILES):,d}')

源语料文件数：1


## 文件采样

由于只是用于评估，所以只使用很少的文件

In [18]:
# K = 5

# print(f'选取 {K} 个输入语料文件')

# SRC_FILES = sorted(random.choices(INPUT_FILES, k=5))

# SRC_FILES

### 统计输入文件总行数

In [19]:
%%time

total_lines = text_files_line_count(INPUT_FILES)
print(f'源语料行数：{total_lines:,d}')



源语料行数：5,398
CPU times: user 70.8 ms, sys: 53.2 ms, total: 124 ms
Wall time: 273 ms


## 处理

我们目前的评估目标

1. 进行人工评估，输出列表文件进行比对
1. 输入文字为回答数据的逗号前的半句

### 处理函数定义

从段落中拆出开头的几句作为预测上下文

In [20]:

def proc_line(line):
    result = []
    line = line.strip()
    if not line:
        return result
    min_length = MIN_LEN
    max_length = MAX_LEN
    paragraphs = json.loads(line)
    for sentences in paragraphs:
        population = []
        weights = []
        for i, v in enumerate(accumulate(len(SP.encode_as_ids(sent)) for sent in sentences[:-1])):
            if min_length < v <= max_length:
                population.append(i)
                weights.append(math.log2(v))
        if population:
            elems = random.choices(population, weights=weights)
            i = elems[0]
            d = {
                'text': ''.join(sentences[:i+1]),
                'infra': ''.join(sentences[i+1: ]),
            }
            result.append(d)
        
    return result


    

### 输出文件 tsv/json

In [21]:
OUTPUT_TSV_FILE = '../data/eval.tsv'
OUTPUT_JSON_FILE = '../data/eval.json'

In [22]:

n_samples = 0

lines_iterator = text_files_line_iterator(INPUT_FILES)
with Pool() as pool, \
     open(OUTPUT_TSV_FILE, 'w') as fp_tsv, \
     open(OUTPUT_JSON_FILE, 'w') as fp_json:
    writer = csv.DictWriter(
        fp_tsv, delimiter='\t', quoting=csv.QUOTE_MINIMAL,
        fieldnames=['text', 'infra']
    )

    it = pool.imap_unordered(
        proc_line,
        tqdm(lines_iterator, 'map lines', total=total_lines),
        chunksize=512
    )
    for result in tqdm(it, 'reduce all', total=total_lines):
        for d in result:
            writer.writerow(d)
            print(json.dumps(d, ensure_ascii=False), file=fp_json)
            n_samples += 1

print(f'得到语料样本数：{n_samples:,d}')




得到语料样本数：40,717


In [23]:
os.path.splitext('eval.out.json')

('eval.out', '.json')